In [40]:
import os
# os.chdir('../')
%pwd

'c:\\Users\\LENOVO\\OneDrive\\Desktop\\complete_end_to_end_project'

In [41]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class Datatrainer_config:
    root_dir: Path
    train_data: Path
    test_data: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_col: str

In [42]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yml,create_directories

class Data_trainer_config_manger:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH,schema_file_path=SCHEMA_FILE_PATH):
        self.config=read_yml(config_file_path)
        self.schema=read_yml(schema_file_path)
        self.params=read_yml(params_file_path)

        create_directories([self.config.artifacts_roots])

    def get_data_trainer_config(self)->Datatrainer_config:
        config=self.config.data_trainer
        schema_col=self.schema.TARGET_COLUMNS
        params=self.params.ELASTIC_NET
        create_directories([config.root_dir])

        data_trainer_config=Datatrainer_config(
                root_dir=config.root_dir ,
                train_data= config.train_path,
                test_data= config.test_path,
                model_name= config.model_name,
                alpha=params.l1_ratio,
                l1_ratio=params.alpha,
                target_col=schema_col.NAME          
        )   

        return data_trainer_config



        

In [43]:
import pandas as pd
import os
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib

class Data_trainer:
    def __init__(self,config:Datatrainer_config):
        self.config=config

    def model_training(self):
        train_data=pd.read_csv(self.config.train_data)
        test_data=pd.read_csv(self.config.test_data)

        x_train=train_data.drop(columns=self.config.target_col)
        x_test=test_data.drop(columns=self.config.target_col)

        y_train=train_data[[self.config.target_col]]
        y_test=test_data[[self.config.target_col]]

        model=ElasticNet(l1_ratio=self.config.l1_ratio,alpha=self.config.alpha,random_state=42)
        model.fit(x_train,y_train)

        joblib.dump(model,os.path.join(self.config.root_dir,self.config.model_name))


        


In [44]:
try: 
    
    confif_manager=Data_trainer_config_manger()
    config=confif_manager.get_data_trainer_config()
    data_transformation=Data_trainer(config)    
    data_transformation.model_training()
except Exception as e:
    raise e


[2025-02-01 22:57:42,420: INFO: common: yaml file config\config.yaml loaded succesfully]
[2025-02-01 22:57:42,427: INFO: common: yaml file schema.yaml loaded succesfully]
[2025-02-01 22:57:42,431: INFO: common: yaml file params.yaml loaded succesfully]
[2025-02-01 22:57:42,433: INFO: common: created directory at path artifacts]
[2025-02-01 22:57:42,434: INFO: common: created directory at path artifacts/data_trainer]
